In [1]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2


In [2]:
em = pd.read_excel("../data/cleaned/emdat_cleaned_final.xlsx")
kg = pd.read_excel("../data/cleaned/kaggle_cleaned_magnitude4.xlsx")

em.head(), kg.head()


(   Magnitude  Latitude  Longitude  Total Deaths       Date Location
 0        5.3    39.700     42.800            50 1924-05-13  ERZURUM
 1        6.8    39.897     41.891            60 1924-09-13  ERZURUM
 2        5.8    41.200     42.800           200 1925-01-09  ARDAHAN
 3        5.9    38.000     30.500             3 1925-08-07    DINAR
 4        NaN    38.250     27.100            50 1928-03-30    IZMIR,
    Index       Date         Time_raw  Latitude  Longitude  Depth  \
 0      2 2023-12-31  17:06:05.420000   37.4405    43.9263    5.0   
 1      4 2023-12-31  16:37:55.840000   37.4763    43.6965    5.0   
 2      6 2023-12-30  13:26:24.150000   38.4627    39.0540    5.0   
 3      7 2023-12-30  02:05:13.040000   36.4303    36.3330   12.3   
 4      8 2023-12-28  10:40:34.560000   36.4075    28.3622   28.0   
 
                                           Location  Year  Month  Day  \
 0  YAZILI-YUKSEKOVA (HAKKARI) [SOUTH WEST  3.6 KM]  2023     12   31   
 1          CIMENLI- (H

In [3]:
#Bu fonksiyon iki coğrafi nokta arasındaki gerçek km mesafesini hesaplar.
#Merge’de coğrafya kriteri için kullanacağız: 50 km içinde eşleşme kabul edilir
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Dünya yarıçapı (km)
    
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    return R * c



In [8]:
def find_match(row, kg, day_range, distance_limit):
    date_low  = row["Date"] - pd.Timedelta(days=day_range)
    date_high = row["Date"] + pd.Timedelta(days=day_range)
    
    # Tarih filtresi
    subset = kg[(kg["Date"] >= date_low) & (kg["Date"] <= date_high)]
    if subset.empty:
        return None
    
    # Mesafe filtrele
    subset = subset.copy()
    subset["distance"] = subset.apply(
        lambda x: haversine(row["Latitude"], row["Longitude"], x["Latitude"], x["Longitude"]),
        axis=1
    )
    
    subset = subset[subset["distance"] < distance_limit]
    if subset.empty:
        return None
    
    # Magnitude toleransı: ±0.5
    if not pd.isna(row["Magnitude"]):
        subset = subset[abs(subset["Magnitude"] - row["Magnitude"]) <= 0.5]
    
    if subset.empty:
        return None
    
    return subset.sort_values("distance").iloc[0]


In [9]:
merged_rows = []

for idx, row in em.iterrows():

    # ---------------- STRICT MATCH (±1 gün, 50 km) ----------------
    strict_match = find_match(row, kg, day_range=1, distance_limit=50)
    
    if strict_match is not None:
        merged_rows.append({
            "Date": strict_match["Date"],
            "Time": strict_match["Time"],
            "Magnitude": strict_match["Magnitude"],
            "Depth": strict_match["Depth"],
            "Latitude": strict_match["Latitude"],
            "Longitude": strict_match["Longitude"],
            "Kaggle_Location": strict_match["Location"],
            "EMDAT_Location": row["Location"],
            "Total_Deaths": row["Total Deaths"],
            "EMDAT_Date": row["Date"],
            "Distance_km": strict_match["distance"],
            "match_type": "strict"
        })
        continue

    # ---------------- LOOSE MATCH (±3 gün, 75 km) ----------------
    loose_match = find_match(row, kg, day_range=3, distance_limit=75)
    
    if loose_match is not None:
        merged_rows.append({
            "Date": loose_match["Date"],
            "Time": loose_match["Time"],
            "Magnitude": loose_match["Magnitude"],
            "Depth": loose_match["Depth"],
            "Latitude": loose_match["Latitude"],
            "Longitude": loose_match["Longitude"],
            "Kaggle_Location": loose_match["Location"],
            "EMDAT_Location": row["Location"],
            "Total_Deaths": row["Total Deaths"],
            "EMDAT_Date": row["Date"],
            "Distance_km": loose_match["distance"],
            "match_type": "loose"
        })


In [10]:
merged = pd.DataFrame(merged_rows)
merged.head()


,Date,Time,Magnitude,Depth,Latitude,Longitude,Kaggle_Location,EMDAT_Location,Total_Deaths,EMDAT_Date,Distance_km,match_type
0,1924-09-13,14:34:14.700000,6.3,10.0,39.96,41.94,EMRE-KOPRUKOY (ERZURUM) [SOUTH WEST 1.9 KM],ERZURUM,60,1924-09-13,8.156672,strict
1,1925-01-09,17:38:34.300000,5.7,60.0,41.33,43.41,AKCIL-CILDIR (ARDAHAN) [EAST 20.4 KM],ARDAHAN,200,1925-01-09,52.994229,loose
2,1925-08-07,06:46:37.000,5.6,20.0,38.10,29.80,BOZDAG-CIVRIL (DENIZLI) [WEST 2.2 KM],DINAR,3,1925-08-07,62.294393,loose
3,1928-03-31,05:12:24.000,5.1,30.0,38.10,27.40,SAGLIK-TORBALI (IZMIR) [NORTH 1.6 KM],IZMIR,50,1928-03-30,31.078817,strict
4,1938-04-19,10:59:20.400000,6.2,10.0,39.44,33.79,YUKARICIFTLIKKOYU-KAMAN (KIRSEHIR) [EAST 2.9 KM],KIRSEHIR,149,1938-04-19,10.330058,strict


In [12]:
merged.shape

(54, 12)

In [13]:
merged.head(20)

,Date,Time,Magnitude,Depth,Latitude,Longitude,Kaggle_Location,EMDAT_Location,Total_Deaths,EMDAT_Date,Distance_km,match_type
0,1924-09-13,14:34:14.700000,6.3,10.0,39.96,41.94,EMRE-KOPRUKOY (ERZURUM) [SOUTH WEST 1.9 KM],ERZURUM,60,1924-09-13,8.156672,strict
1,1925-01-09,17:38:34.300000,5.7,60.0,41.33,43.41,AKCIL-CILDIR (ARDAHAN) [EAST 20.4 KM],ARDAHAN,200,1925-01-09,52.994229,loose
2,1925-08-07,06:46:37.000,5.6,20.0,38.10,29.80,BOZDAG-CIVRIL (DENIZLI) [WEST 2.2 KM],DINAR,3,1925-08-07,62.294393,loose
3,1928-03-31,05:12:24.000,5.1,30.0,38.10,27.40,SAGLIK-TORBALI (IZMIR) [NORTH 1.6 KM],IZMIR,50,1928-03-30,31.078817,strict
4,1938-04-19,10:59:20.400000,6.2,10.0,39.44,33.79,YUKARICIFTLIKKOYU-KAMAN (KIRSEHIR) [EAST 2.9 KM],KIRSEHIR,149,1938-04-19,10.330058,strict
5,1939-09-22,00:36:36.600000,6.2,10.0,39.07,26.94,KIZILCUKUR-DIKILI (IZMIR) [SOUTH EAST 4.2 KM],DIKILI,60,1939-09-22,24.740677,strict
6,1940-02-21,00:50:00.000,5.1,30.0,38.40,35.30,CAYIROZU-DEVELI (KAYSERI) [SOUTH 2.6 KM],KAYSERI,37,1940-02-20,17.428544,strict
7,1941-09-10,21:53:56.600000,5.6,20.0,39.45,43.32,YANKAYA-TASLICAY (AGRI) [SOUTH WEST 3.3 KM],VAN,430,1941-09-11,5.818729,strict
8,1941-11-12,10:04:59.200000,5.6,70.0,39.74,39.43,YENIKOY- (ERZINCAN) [NORTH EAST 1.5 KM],ERZINCAN,15,1941-11-12,5.134833,strict
9,1942-12-20,14:03:07.800000,6.5,10.0,40.87,36.47,ERDEMLI-ERBAA (TOKAT) [NORTH WEST 2.8 KM],"NIKSAR, ERBAA",3000,1942-12-20,22.030309,strict


In [14]:
len(em) - len(merged)


51

In [15]:
merged.to_csv("../data/merged/merged_strict.csv", index=False)
merged.to_excel("../data/merged/merged_strict.xlsx", index=False)


In [16]:
matched_dates = merged["EMDAT_Date"]
unmatched = em[~em["Date"].isin(matched_dates)]
unmatched.shape


(51, 6)

In [17]:
def fuzzy_match(row, kg):
    # tarih toleransı: ±7 gün
    date_low  = row["Date"] - pd.Timedelta(days=7)
    date_high = row["Date"] + pd.Timedelta(days=7)

    subset = kg[(kg["Date"] >= date_low) & (kg["Date"] <= date_high)]
    if subset.empty:
        return None

    subset = subset.copy()

    # mesafe toleransı: 120 km
    subset["distance"] = subset.apply(
        lambda x: haversine(row["Latitude"], row["Longitude"], 
                            x["Latitude"], x["Longitude"]),
        axis=1
    )
    subset = subset[subset["distance"] < 120]
    if subset.empty:
        return None

    # magnitude toleransı: ±1.0 (fuzzy)
    if not pd.isna(row["Magnitude"]):
        subset = subset[abs(subset["Magnitude"] - row["Magnitude"]) <= 1.0]
    if subset.empty:
        return None

    # confidence score (0-3 arası)
    subset["confidence_score"] = (
        1 / (1 + abs((subset["Date"] - row["Date"]).dt.days)) +
        1 / (1 + subset["distance"] / 50) +
        1 / (1 + abs(subset["Magnitude"] - row["Magnitude"]))
    )

    # en yüksek confidence score'a sahip depremi döndür
    return subset.sort_values("confidence_score", ascending=False).iloc[0]


In [18]:
fuzzy_rows = []

for idx, row in unmatched.iterrows():
    match = fuzzy_match(row, kg)
    if match is not None:
        fuzzy_rows.append({
            "Date": match["Date"],
            "Time": match["Time"],
            "Magnitude": match["Magnitude"],
            "Depth": match["Depth"],
            "Latitude": match["Latitude"],
            "Longitude": match["Longitude"],
            "Kaggle_Location": match["Location"],
            "EMDAT_Location": row["Location"],
            "Total_Deaths": row["Total Deaths"],
            "EMDAT_Date": row["Date"],
            "Distance_km": match["distance"],
            "confidence_score": match["confidence_score"],
            "match_type": "fuzzy"
        })


In [19]:
fuzzy = pd.DataFrame(fuzzy_rows)
fuzzy.head()
fuzzy.shape


(11, 13)

In [20]:
fuzzy.to_csv("../data/merged/merged_fuzzy.csv", index=False)
fuzzy.to_excel("../data/merged/merged_fuzzy.xlsx", index=False)


In [21]:
# Strict ve fuzzy datasetleri birleştir
combined = pd.concat([merged, fuzzy], ignore_index=True)

# Sütunları mantıklı bir sırayla düzenleyelim (opsiyonel)
combined = combined[[
    "Date", "Time", "Magnitude", "Depth", "Latitude", "Longitude",
    "Kaggle_Location", "EMDAT_Location", "Total_Deaths",
    "EMDAT_Date", "Distance_km", "confidence_score", "match_type"
]].sort_values("Date")

# Eksik confidence_score değerlerini strict/loose için dolduralım
combined["confidence_score"] = combined["confidence_score"].fillna(3.0)


combined.shape


(65, 13)

In [22]:
combined.to_csv("../data/merged/merged_all_matches.csv", index=False)
combined.to_excel("../data/merged/merged_all_matches.xlsx", index=False)